In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import numpy as np
def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [ ]:
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df.sort_values(['ID','Date'],inplace=True)
df['year'] = df['Date'].apply(lambda x: x[:4])
df['month&day'] = df['Date'].apply(lambda x: x[5:])
df = df[df['month&day'] != '02-29']

# max and min temperature 2005-->2014

max_2005_to_2014 = df[(df['Element'] == 'TMAX') & (df['year'] != '2015')]
min_2005_to_2014 = df[(df['Element'] == 'TMIN') & (df['year'] != '2015')]

max_2005_to_2014 = max_2005_to_2014.groupby('month&day')['Data_Value'].agg([np.max])
min_2005_to_2014 = min_2005_to_2014.groupby('month&day')['Data_Value'].agg([np.min])

# max and min temp for which decade record was broken in 2015
max_2015 = df[(df['Element'] == 'TMAX') & (df['year'] == '2015')]
min_2015 = df[(df['Element'] == 'TMIN') & (df['year'] == '2015')]

max_2015 = max_2015.groupby('month&day')['Data_Value'].agg([np.max])
min_2015 = min_2015.groupby('month&day')['Data_Value'].agg([np.min])

broken_max = np.where(max_2015['amax'] > max_2005_to_2014['amax'])[0]
broken_min = np.where(min_2015['amin'] < min_2005_to_2014['amin'])[0]


# plotting the final data
plt.figure()
plt.plot(min_2005_to_2014.values,'b',label = 'Mininmum Temperatures from 2005 to 2014')
plt.plot(max_2005_to_2014.values,'y',label = 'Maximum Temperatures from 2005 to 2014')
plt.scatter(broken_min,min_2015.iloc[broken_min],s=20, c='black', label='The Broken Minimum')
plt.scatter(broken_max,max_2015.iloc[broken_max],s=20, c='m', label='The Broken Maximum')

plt.gca().fill_between(range(len(min_2005_to_2014)),
                             min_2005_to_2014['amin'],max_2005_to_2014['amax'],
                             facecolor='g',
                             alpha=0.25)
plt.xticks(range(0, len(min_2005_to_2014), 20),min_2005_to_2014.index[range(0, len(min_2005_to_2014), 20)], rotation = '45')
plt.gca().axis([-7, 370, -215, 650])

plt.xlabel('year days')
plt.ylabel('Temperature in °F')
plt.title('Tempertaure Variation from 2005 to 2014\nAnn Arbor, Michigan, United States')
plt.legend(loc = 0, frameon = False)
plt.tick_params(top=False, bottom=True, left=False, right=False, labelleft=True, labelbottom=True)

for spine in plt.gca().spines.values():
    spine.set_visible(False)

plt.show()